# Chatbot Assistant

**Conversational system prompt:** friendlier tone, acknowledgments, natural follow-ups.

RAG from a single URL + memory + Gradio. Same logic as base; only the **system prompt** is changed so the agent is explicitly instructed to be conversational (acknowledge, clarify, natural flow).

## Assignment — Building Applications with LLMs & Agents

**Source:** *Assignment - Building Applications with LLMs & Agents.pdf* (read in the cell below).

### Assignment summary
- **Business problem:** Build a virtual assistant for a client who provides a URL; the chatbot answers questions **only from the content on that website**.
- **ML problem:** RAG system with: (1) **Data source** — single webpage (use LangChain loader), (2) **Vector store** for document representations, (3) **LLM** to answer user questions.
- **Tasks:** Chunk input (e.g. RecursiveCharacterTextSplitter), generate embeddings (OpenAIEmbeddings or Sentence Transformers), store in a vector store (e.g. FAISS), prompt the LLM to use retrieval as knowledge.
- **Extra (conversational):** User can ask follow-ups; e.g. "I want to purchase an iPhone" → system lists iPhone 13, 14, 15 → user asks "What is the price of the oldest one" and the system should understand that means the oldest of those (e.g. iPhone 13) without the user re-typing.
- **Deliverables:** Solution in a notebook; optional Gradio UI.

---

### What has been accomplished

| Requirement | Status | Implementation in this notebook |
|-------------|--------|---------------------------------|
| Single webpage as data source | Done | **Section 2:** `WebBaseLoader(URL)` loads the client URL (example: LangChain RAG docs). |
| Chunk document to fit model length | Done | **Section 3:** `RecursiveCharacterTextSplitter` (chunk_size=1000, overlap=200). |
| Generate embeddings | Done | **Section 3:** `OpenAIEmbeddings()`; chunks embedded before storage. |
| Store in vector store | Done | **Section 3:** `FAISS.from_documents(chunks, embeddings)`. |
| Prompt LLM to use retrieval | Done | **Section 4:** RAG agent with `retrieve_context` tool (FAISS retriever); prompt instructs use of retrieved context. |
| Conversational / multi-turn | Done | **Section 4:** `chat_history` in prompt; `get_chat_history` tool; full history passed each turn so follow-ups (e.g. "the oldest one") are understood. |
| Follow-up "price of the oldest one" | Done | **Section 4:** System prompt and `get_chat_history` description explicitly require using conversation first for "the oldest", "that one", etc., and answering from the list already given (no new search). |
| Gradio UI | Done | **Section 5:** `gr.ChatInterface` with multi-turn `predict()` that uses the agent and maintains session history. |
| Read assignment file | Done | Cell below reads the assignment PDF and displays its text. |

In [ ]:
# Read assignment PDF and display its content (optional: set path if the file is elsewhere)
import os
from pathlib import Path

# Look for assignment PDF: env ASSIGNMENT_PDF_PATH, or current dir, or set path explicitly below
assignment_path = os.environ.get("ASSIGNMENT_PDF_PATH", "Assignment - Building Applications with LLMs & Agents.pdf")
assignment_path = Path(assignment_path)
if not assignment_path.is_absolute():
    assignment_path = Path.cwd() / assignment_path

def read_assignment_pdf(path):
    try:
        from pypdf import PdfReader
        reader = PdfReader(str(path))
        text_by_page = []
        for i, page in enumerate(reader.pages):
            t = page.extract_text() or ""
            text_by_page.append(f"--- Page {i+1} ---\n{t}")
        return "\n\n".join(text_by_page)
    except ImportError:
        return "Install pypdf to read PDF: pip install pypdf"
    except Exception as e:
        return f"Cannot read PDF: {e}\nPlace the assignment PDF at: {path}"

if assignment_path.exists():
    assignment_text = read_assignment_pdf(assignment_path)
    print(assignment_text[:8000] if len(assignment_text) > 8000 else assignment_text)
    if len(assignment_text) > 8000:
        print("\n... [truncated; full assignment has been read]")
else:
    print(f"Assignment PDF not found at: {assignment_path}")
    print("Set ASSIGNMENT_PDF_PATH to the full path of 'Assignment - Building Applications with LLMs & Agents.pdf' (e.g. in env or in this cell) to read it here.")

## 1. Install dependencies and load environment

In [11]:
!pip install -q python-dotenv langchain langchain-core langchain-community langchain-openai langchain-text-splitters faiss-cpu gradio beautifulsoup4 pypdf

import os
from dotenv import load_dotenv
load_dotenv()

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import create_retriever_tool, tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.tools.tavily_search import TavilySearchResults
import gradio as gr

print("Dependencies loaded. OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))

Dependencies loaded. OPENAI_API_KEY set: True


## 2. Load document from URL (single webpage)

Uses LangChain's **WebBaseLoader** as required (client provides a URL).

In [12]:
# Single webpage: LangChain RAG docs (assignment: "client will give XYZ a URL")
URL = "https://docs.langchain.com/oss/python/langchain/rag"

loader = WebBaseLoader(URL)
docs = loader.load()

print(f"Loaded {len(docs)} document(s). Total chars: {sum(len(d.page_content) for d in docs)}")
if docs:
    print(docs[0].page_content[:500], "...")

Loaded 1 document(s). Total chars: 31414
Build a RAG agent with LangChain - Docs by LangChainSkip to main contentDocs by LangChain home pageOpen sourceSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a RAG agent with LangChainDeep AgentsLangChainLangGraphIntegrationsLearnReferenceContributePythonLearnTutorialsDeep AgentsLangChainSemantic searchRAG agentSQL agentVoice agentMulti-agentLangGraphConceptual overviewsLangChain vs. LangGraph vs. Deep AgentsComponent architectureMemoryContextGraph APIFunctiona ...


## 3. Chunk, embed, and store in FAISS

- **Split**: RecursiveCharacterTextSplitter (fits model context).
- **Embed**: OpenAIEmbeddings.
- **Store**: FAISS vector store.

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""],
)
chunks = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(chunks, embeddings)

print(f"Stored {len(chunks)} chunks in FAISS.")

Stored 42 chunks in FAISS.


## 4. RAG agent with conversational memory

- **Retriever tool**: search the vector store (RAG).
- **Chat history tool**: so the agent can recall previous Q&A (multi-turn).
- **Prompt**: includes `chat_history` so the model sees prior turns.

In [14]:
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Retriever tool (RAG)
retriever = vector_store.as_retriever(search_type="similarity", k=4)
retrieve_tool = create_retriever_tool(
    retriever,
    name="retrieve_context",
    description="Retrieve relevant passages from the website to answer the user. Use when you need to look up information.",
)

# In-memory store for chat history (per session)
store = {}
current_session_id = "default"

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

@tool
def get_chat_history() -> str:
    """Return the recent conversation history (previous questions and answers). Always use when the user refers to something from the current conversation (e.g. 'the oldest one', 'the first one', 'that one', 'its price', 'the one you just said', 'which was the cheapest') so you can find the exact list or details you already gave and answer from that instead of searching again."""
    if current_session_id not in store:
        return "No previous messages."
    hist = store[current_session_id]
    parts = []
    for m in hist.messages:
        if isinstance(m, HumanMessage):
            parts.append(f"User: {m.content}")
        elif isinstance(m, AIMessage):
            parts.append(f"Assistant: {m.content}")
    return "\n".join(parts[-20:]) if parts else "No previous messages."

tavily_search = TavilySearchResults()
tools = [retrieve_tool, get_chat_history, tavily_search]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly, conversational assistant. Use retrieve_context for the loaded website and tavily_search for general or up-to-date web info when needed.\n\nWhen the user asks about something you already mentioned (e.g. 'the oldest', 'the first one', 'that one', 'what was the price of that?', 'the one you just listed'), you MUST use the recent conversation: call get_chat_history, find the list or details from your previous reply, identify which item they mean (e.g. 'oldest' = earliest by release date among those), and answer from that—do not run a new search. Only use retrieve_context or tavily_search when the answer is not in the conversation. Prefer the website context when the question is about the loaded page; otherwise use the conversation first, then web search if needed. Acknowledge what they said when relevant. Keep answers clear and concise but natural."),

    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

print("RAG agent with memory ready.")

RAG agent with memory ready.


## 5. Gradio deployment (conversational UI)

Multi-turn: each user message is sent with full `chat_history`; after the agent replies we append the turn to history so follow-up questions are understood.

In [15]:
def predict(message, history):
    """Gradio chat handler: conversational (multi-turn) with agent memory."""
    if not message or not message.strip():
        return ""
    
    current_session_id = "default"  # or derive from Gradio user/session
    hist = get_session_history(current_session_id)
    chat_history = hist.messages
    
    try:
        result = agent_executor.invoke({
            "input": message.strip(),
            "chat_history": chat_history,
        })
        answer = result["output"]
    except Exception as e:
        answer = f"Error: {e}"
    
    hist.add_user_message(message.strip())
    hist.add_ai_message(answer)
    
    return answer

demo = gr.ChatInterface(
    fn=predict,
    title="RAG Chatbot (URL + Conversational)",
    description="Ask questions about the loaded webpage. You can ask follow-ups (e.g. 'What about the first one?').",
    chatbot=gr.Chatbot(height=400),
    textbox=gr.Textbox(placeholder="Ask about the page content...", container=False, scale=7),
    examples=["What is RAG?", "What is task decomposition?"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


/Users/satta/Library/Python/3.9/lib/python/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(




> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'iPhone price'}`


[{'url': 'https://www.verizon.com/smartphones/apple/', 'content': '## Apple iPhone 16 Plus\n\nStarts at $23.05/mo\n\nfor 36 months, 0% APR\n\nfor36months, 0% APR\n\nRetail price: $829.99\n\nCustomize Colors for Apple iPhone 16 Plus\n\n Black\n White\n Pink\n Ultramarine\n Teal\n\n## Apple iPhone 14 Pro\n\nStarts at $24.99/mo\n\nfor 36 months, 0% APR\n\nfor36months, 0% APR\n\nRetail price: $899.99\n\nCustomize Colors for Apple iPhone 14 Pro\n\n Space Black\n Gold - Out of stock\n Deep Purple - Out of stock\n\n## Apple iPhone 16 Pro Max\n\nStarts at $30.55/mo\n\nfor 36 months, 0% APR\n\nfor36months, 0% APR\n\nRetail price: $1,099.99\n\nCustomize Colors for Apple iPhone 16 Pro Max\n\n Black Titanium - Out of stock\n White Titanium - Out of stock\n Desert Titanium - Out of stock\n Nat